<h1 align='center'> COMP2420/COMP6420 - Introduction to Data Management, Analysis and Security</h1>

<h2 align='center'> Lab 08 - Databases</h2>

*****

During the lectures, you have learned about **SQL (Structured Query Language)**, a database query language which can be used to create and modify databases. The commands used for such operations can be categorised into  
* **DDL (Data Definition Language)** - Commands used to define database schema 
    * **CREATE** – Used to create the database or its objects.
    * **DROP** – Used to delete objects from the database.
    * **ALTER** - Used to alter the structure of the database.
    * **TRUNCATE** – Used to remove all records from a table (including spaces allocated for the records).
    * **RENAME** – Used to rename an object existing in the database.
* **DML (Data Manipulation Language)** - Commands used to manipulate data present in the database
    * **SELECT** – Used to retrieve data from the database.
    * **INSERT** – Used to insert data into a table.
    * **UPDATE** – Used to update existing data within a table.
    * **DELETE** – Used to delete records from a database table.
* **DCL (Data Control Language)** - Commands used to grant permissions and access rights in the database
    * **GRANT** - Gives users access privileges to database.
    * **REVOKE** - Withdraws users' access privileges to database.

Let's try to implement some of these operations in a PostgreSQL database using Python's _**psycopg2 module**_.

In [1]:
# Import Required Modules
import psycopg2         # Module to interface between Python and PSQL
import pandas as pd     # Improves readability by using dataframes to display query results

def execute_sql(query):
    """ Executes the provided query string (SELECT only)
        on the database currently connected to by the
        global psycopg2 cursor (must be defined namespace
        for which this function has scope).
        
        Returns the result of this select query (should
        there be any) as a pandas dataframe
    """
    # Select table and display
    curr.execute(query)

    # Fetches all the rows from the result of the query
    rows = curr.fetchall()
    
    # Gets the column names for the table
    colnames = [desc[0] for desc in curr.description]

    # Converts into readable pandas dataframe
    df_result = pd.DataFrame(rows, columns=colnames)
    return df_result

To practice our SQL statements we need to create a database on which we can run our queries. The easy way to create the whole database is by just running an SQL script that creates the whole database and constituent tables. 

To connect to a PostgreSQL database and execute SQL commands using Python, we require several functions from the **psycopg2 module**. To connect to a PostgreSQL database, the **connect()** function is used. The following is the list of the connection parameters that are to be passed with this function:

* **database**: the name of the database that you want to connect.
* **user**: the username used to authenticate.
* **password**: password used to authenticate.
* **host**: database server address e.g., localhost or an IP address
* **port**: the port number (that defaults to 5432 if it is not provided)   

The connect() function creates a new database session and returns a new instance of the connection class. With a connection object, you can create a new cursor (with **cursor()** function) to execute an SQL statement (with **execute()** function) and terminate a transaction using either commit() or rollback() method.

In [5]:
### EDIT: Replace uXXXXXXX with your UID
uid = "uXXXXXXX"

In [21]:
# Connect using psycopg2
### EDIT: Replace uXXXXXXX with your UniID
conn = psycopg2.connect(dbname=uid, host="/var/run/postgresql")

# Activate connection cursor
curr = conn.cursor()

# Open and run the SQL script that creates the DB schema
# NOTE: If you recieve the error "no schema has been selected to create in",
#       you need to create a schema for your account:
#       CREATE SCHEMA uXXXXXXX;
sqlfile = open('db_schema.sql', 'r')
curr.execute(sqlfile.read())

The database that we just created looks somewhat like this - 

* **players** (**player_id**: integer, **name** : varchar(50), **position** : varchar(10), **height** : integer, **weight** : integer, **team**: varchar(30))

Each player is assigned a unique _player_id_. The position of a player can either be **Defender**, **Midfielder** or **Forward**. The height of a player is in centimeters while the weight is in kilograms. Each player plays for only one team. The _team_ field is a foreign key to the table **team**.

* **team** (**name**: varchar(30), **city**: varchar(20))

Each team has a **unique name** associated with it. There can be multiple teams from the same city.

* **game** (**game_id**: integer, **home_team**: varchar(30), **away_team** : varchar(30), **home_score** : integer, **away_score** : integer)

Each game has a unique _game_id_. The fields **home_team** and **away_team** are foreign keys to **team**. Two teams may play each other multiple times each season. There is an integrity check to ensure home_team and away_team are different.

* **game_stats** (**player_id** : integer, **game_id**: integer, **goals** : integer, **assists** : integer, **passes** : integer)

game_stats records the performance statistics of a player within a game. A player may not play in every game, in which case it will not have its statistics recorded for that game. **game_id** is a foreign key to game. **team** is a foreign key to team. Assume that there is already an integrity check to ensure that the player involved belongs to either the involving home or away teams.

Now that we know how the database schema looks like, let's see the data rows inside each table - 

In [7]:
# Select table and display
curr.execute("SELECT * FROM team;")

# Fetches all the rows from the result of the query
rows = curr.fetchall()

# Gets the column names for the table
colnames = [desc[0] for desc in curr.description]

# Converts into readable pandas dataframe
df_team = pd.DataFrame(rows, columns=colnames)
df_team

## We convenietly wrap all this functionality into a function 
## named execute_sql(query_string) defined above.

,name,city
0,Arsenal,London
1,Liverpool,Liverpool
2,Chelsea,London
3,Manchester United,Manchester
4,Manchester City,Manchester
5,Tottenham Hotspurs,London


Here, we could have just displayed the rows returned by the **execute() method** using just the first 2 lines of code. However, the advantage of using a pandas dataframe to display the resultant rows is -

* The data is in a more readable and natural table-like format.
* We can perform our usual data wrangling and transformation on this dataframe easily.

In [8]:
df_player = execute_sql("SELECT * FROM player;")
df_player

,player_id,name,position,height,weight,team
0,1,Alexandre Lacazette,Forward,175,73,Arsenal
1,2,Mohamed Salah,Forward,175,71,Liverpool
2,3,Marcos Alonso,Defender,188,85,Chelsea
3,4,Paul Pogba,Midfielder,191,84,Manchester United
4,5,David Silva,Midfielder,173,67,Manchester City
5,6,Jan Vertonghen,Defender,189,88,Tottenham Hotspurs


In [9]:
df_game = execute_sql("SELECT * FROM game;")
df_game

,game_id,home_team,away_team,homescore,awayscore
0,12,Arsenal,Liverpool,0,3
1,23,Liverpool,Chelsea,2,1
2,34,Chelsea,Manchester United,3,1
3,45,Manchester United,Manchester City,1,5
4,56,Manchester City,Tottenham Hotspurs,2,0
5,61,Tottenham Hotspurs,Arsenal,2,2


In [10]:
df_game_stats = execute_sql("SELECT * FROM game_stats;")
df_game_stats

,player_id,game_id,goals,assists,passes
0,2,12,2,0,80
1,2,23,1,1,70
2,3,34,0,1,67
3,4,45,2,2,71
4,5,56,1,0,59
5,1,61,1,1,62


Now, let's try a different type of SQL query. An **INSERT** to insert a new record in one of the tables. This query is also executed with the **execute()** function, however, it also requires a **commit()** function to commit the transaction to  the database.  

In [11]:
# Execute insert query
curr.execute("INSERT INTO player VALUES(7, 'Aaron Ramsey', 'Midfielder', 178, 76, 'Arsenal')")
conn.commit()
# DO NOT RUN THIS CELL MORE THAN ONCE

In [12]:
# Fetching all records from the table again
df_player = execute_sql("SELECT * FROM player;")
df_player

,player_id,name,position,height,weight,team
0,1,Alexandre Lacazette,Forward,175,73,Arsenal
1,2,Mohamed Salah,Forward,175,71,Liverpool
2,3,Marcos Alonso,Defender,188,85,Chelsea
3,4,Paul Pogba,Midfielder,191,84,Manchester United
4,5,David Silva,Midfielder,173,67,Manchester City
5,6,Jan Vertonghen,Defender,189,88,Tottenham Hotspurs
6,7,Aaron Ramsey,Midfielder,178,76,Arsenal


Similarly, we can also delete rows from a table by executing the **DELETE** SQL statement on the particular table. **Always remember to specify the condition which is unique to the row you want to delete (here, PRIMARY KEY is used most often to grab the unique identity of a record)**, otherwise, you might end up deleting more than one rows in the table. For example, 

**DELETE FROM player WHERE player_id = 6;**

will only delete the record where the player_id is 6.

And, finally, when we have to close the connection to the database, we call the **close()** function on the cursor as well as the connection.

In [13]:
# Close connection
curr.close()
conn.close()

<br/>

## Exercise 1 - Who Am I?

Now that we know how to work with SQL using Python, let's apply what we have learned in the lectures, and flex our SQL muscles to come up with queries that can answer the following questions -  

In [14]:
# Reconnecting to the database using psycopg2
conn = psycopg2.connect(dbname=uid, host="/var/run/postgresql")

# Reactivating connection cursor
curr = conn.cursor()

### 1. List all the players who are taller than 175cm in height.

In [4]:
# YOUR ANSWER HERE


### 2. List all teams (in alphabetical order) and game scores where they have won with a goal margin of 2 goals or more at home. 

In [3]:
# YOUR ANSWER HERE


### 3. List the average height and average weight _(rounded to 2 digits after decimal)_ of the players corresponding to each playing position.

In [2]:
# YOUR ANSWER HERE


### 4. List all the players, their position and their number of passes in any game where those players have scored at least one goal and one assist in a game.

In [1]:
# YOUR ANSWER HERE


<br/>

## Exercise 2 - Patty’s Playschool

**Patty’s Playschool**, a child daycare centre, has offered your family free lifetime childcare if you design their database. Given the very high cost of child care, you agree. Here’s the information that was gathered:

* **Parents** are identified by their government issued ID number, and have a name, address(es), and phone number(s).
* **Children** have a name, and a date of birth.
* Children are registered for the daycare by a parent. Their registration includes other associated information such as allergies, date of registration, (optionally) date of expiry, and the days of the week they are in daycare, and other associated details.
* Each child has a **Category** in which they belong (e.g. infant, toddler). A category is identified by a name, a minimum age, and a maximum age. The category also specifies the maximum number of children that may be looked after at a time by a single staff member.
* **Rooms** in the daycare are identified by a room number, and are designed to cater to a specific category of children.
* **Staff Members** are identified by their staff ID, and have a name, date of birth, address(es), phone number(s).
* Each staff member is allocated to exactly one room. The number of staff members allocated to a room is determined by the number of a children of that category, assigned to that room.

For this exercise, your first task is to draw an **E-R Model** for the data described above. You can work in groups of 2-3 students, and model this database on paper (you'll be provided A3 sheets by the tutors) or use the lab machines to design the model on **draw.io**, **Umbrello**, or **Terra ER**. Please state all assumptions you make which are not stated in the problem. Make sure you specify participation constraints as much as possible. Consider what constraints aren't expressible in E-R modelling, and how they might be implemented in a SQL DDL (if possible, not all are!). Complicated constraints often require additional tools such as database triggers (https://www.postgresql.org/docs/11/plpgsql-trigger.html)

There are many possible ways to model the information presented above. Compare and contrast different options with those who you are working with and those around you. Consider the effects of your model on any potential implementation of this database, and think about what your modelling decisions might imply for any user querying the database that results from your design.


### Task 1 - Design an E-R Diagram to model the above problem
* Feel free to save the resulting image produced by aforementioned tools in this notebook's directory and display it here, by using a HTML image tag:

eg: <verbatim><img src\='./my_er.png'\></verbatim>




**Good Job!**

Now, let's extend this exercise further by making an SQL database from this E-R Diagram. Open a text editor to make an SQL script. In this script, make the DDL statements for your model. Include as much detail from the E-R Diagram as you can, and make sure you specify the constraints and cardinality of relationships.

In [ ]:
# Task 2 - Design the DDL structure of the database based on the 
# E-R Diagram that you designed in Task 1.  
# YOUR ANSWER HERE




**Well done!**

Now, as a way to confirm the correctness of your work, follow the procedure you learnt earlier in this lab to load this SQL script into PSQL using the psycopg2 module, and INSERT one record into each of the tables in your database. 

In [4]:
# Task 3 - Load and run the SQL script that creates the daycare database.
#        - Run SQL commands to insert atleast one record in each table.
# YOUR ANSWER HERE


